# Extrai informações de ATAS

## As ATAs foram obtidas através de validação manual (mais confiável)
## Uma melhoria será automatizar o processo, ie, identificar as ATAs e extrair as informações

# Depedências

In [123]:
import pandas as pd
import numpy as np
import json
import bz2
import argparse
import pdfplumber
import os
import re
from enum import Enum
from unidecode import unidecode
from collections import Counter

# ATAS identificadas e validadas manualmente

In [61]:
docs_atas = {
    'coqueiral': ['2ba108ce36bf3fe2a98daa28419232e3', 'f235a370910ef694a7b511e50decc61d', 'f67e0dccc82d438630d12be9a0f6dbef', 'db1a383dc25f181e3cb87d09da74f2ed',                       '6aa821e1e916cac34caf106284ff38b8', '795030e4733503b546ca971e1d8ecebb', '48f1e627f82574cf25578c815da4637b', '1c4dec7f2fcfda38665e661919e0b124',                       'f459131d918482cde60e72cf7740a203', '7e37c242d98240af4deeab78518fc9e6', '9b031c8b2f8aba5994af276f9507c9c1', '9b0b62c6d4be49cedf64dfa5aedac04b',                       '747efd2fb6d3d9c25da8e73c85cfd3ac', '43c7358e4129a6f927df8dcffe22f884', '5d3064bda50bd93b9f5f5628e1509116', '42ee4c843a96a0d768ff5a477acf5992',                       '933ba4fa37a19c3b37971fadcfd8752e', 'ff48c8fc20d968e4985064b0d0b66bc0', '940a3ab677d4c027a3739522dcdaf42b', '42eb53256bc641a61e1f6978d81b3869'], 
    'cristais': ['f1287cc8fcccf899d0b5d7446381eb34', '499fa518c7724a6e449c509b7d7bc819', '2cb3f9bdfa78bee9de9937c2a8c42174', '1a0df92c6a295bd4ea4776a4e61c9bc8',                       'ea9f6170e573536f8ecbf073a8cce337', '67dfdf460e020cbafa3d5dcb42153132', 'b811b453d6ce70cb0bfc588161b96dd2', '94cbb7c0710eda204d32940b26503796',                       '5a1d3235d4893a0e166fc87b4f30de5b', 'dd9ac6836d95f5c7d620020fb7f86561', '7920f287fb2544648ce2073ebc2deccd', '8536c092ea2d9aa0e826e88d3a1335c4',                       'dbd06aba9f943c1dfda6dd3e05f283ee', '6dd12515afd2ccf77deb66e2d96d4da9'], 
    'passa-vinte': ['5ba3e9d87f8d70702c552c95e95520c0', '7e0a1d03c293b71a0e72c76bd333f7fe', '5ba3e9d87f8d70702c552c95e95520c0', '5ba3e9d87f8d70702c552c95e95520c0',                       '8966befe922f586c17fb62243f0119d3', '848d2567bf35652321bade880bd5561e'], 
    'pirapetinga': ['2d0b02df41a79f669ae2f4b99d672b49', '1f8d824ddb88ae8f8ad6a81ee667e9ef', 'bc7d07419c3eb4df8c3cbf0b3faa1ea0', 'bc7d07419c3eb4df8c3cbf0b3faa1ea0',                       '9f570604176b0196464a654643b7f5df', 'b67877144aeb0a59f6e63ecf9d914bc1', 'b67877144aeb0a59f6e63ecf9d914bc1', 'b67877144aeb0a59f6e63ecf9d914bc1',                       '00f24638fb408897e5ee166907a89db8', '88819c18e27860449b0162bef68d4a68', '88819c18e27860449b0162bef68d4a68', '2c892bb52aa3ba1d158473820673c636',                       '7228fb0214d8ce043a657df41dd316b8', '7228fb0214d8ce043a657df41dd316b8', '9719766f3abb916bd3937cd05c81b34e', 'bbd89bca24b1fe7bcf7b791a7d4767ea',                       'bbd89bca24b1fe7bcf7b791a7d4767ea', '4c3777c684dfca6171414a06a0f03279', '4c3777c684dfca6171414a06a0f03279'], 
    'sao-bento-abade': ['85c1a53d38e9f0128f437554523bf87c', 'c7d2371dbc67b51c8e350e03d0c46b8a', 'c7d2371dbc67b51c8e350e03d0c46b8a',    '08b43f26baa37068d9128b9b248c1c6b', '580e6393365c2957e221de56844dd011', '580e6393365c2957e221de56844dd011', '4d6035bf3212568e336fe0ab7d9fb215',          '2309298b1878b216dca666aba446a4cd', '9fc40f9997abab8da10a91416d645a05', 'c43945c062d0e37751d8850ec2c94f7c', '275acd244d5791013bd8f4b3243a4a1a', 'a58ad8575985794e945075b9955f6823', 'f80ced0ea740516c3d4c57ba4a7805dd']
}

### Função para listagem dos arquivos de PDF a partir do *file descriptor*

In [38]:
# funções úteis
def list_by_file_format(path, file_format):
    path = path.rstrip("/") + "/"
    file_descriptor = [json.loads(line) for line in open(path + "file_description.jsonl").readlines()]
    files = [item["file_name"] for item in file_descriptor if item["type"] in file_format]
    return [f"{path}{f}".rstrip(";") for f in files]

def list_pdf(path):
    return list_by_file_format(path, ["application/pdf", "pdf", "pdf;"])

def beauty_print(grouped):
    for city, processes in grouped.items():
        print(city + ":")
        total = 0
        for no_process, documents in processes.items():
            total += len(documents)
            print(f"\t{no_process:>8s}: {len(documents)}")
        print(f"\t{'total':>8s}: {total}")
        
#  lista arquivos de licitações
base_path = "../data/"
pattern = "\\d+[-_]licitacoes-(([a-z]|-)+)"

sel_path = os.listdir(base_path)
sel_path = list(filter(lambda f: re.match(pattern, f), sel_path)) # Seleciona os que iniciam com <cod>-licitacoes-<cidade>
cities = [re.match(pattern, p)[1] for p in sel_path if not p.endswith("bh")] # Remove BH da lista

# arquivos por cidade
files = {city: list_pdf(f"{base_path}{p}/data/files/")  for city, p in zip(cities, sel_path)}
print("cities: " + ", ".join(files.keys()))
print([(key, len(docs)) for key, docs in files.items()])

False
False
True
False
False
False
False
False
cities: sao-bento-abade, olaria, coqueiral, cristais, pirapetinga, passa-vinte, arantina, ijaci
[('sao-bento-abade', 232), ('olaria', 42), ('coqueiral', 1528), ('cristais', 1736), ('pirapetinga', 1007), ('passa-vinte', 395), ('arantina', 983), ('ijaci', 451)]


### Load assets

In [368]:
# lista de nomes populares brasileiros segundo o IBGE (Censo Demográfico de 2010)
# repositório: https://github.com/datasets-br/prenomes
df = pd.read_csv("assets_m03_extracao_informaoes_atas/nomes.csv")
df = df.loc[((df.ate2010.notna()) & (df.ate2010 >= 20)) | ((df.ate2000.notna()) & (df.ate2000 >= 20))]
print(df.head())
nomes = df.nome.apply(lambda name: str(name)[0].upper() + str(name)[1:]).to_list()

      nome   ate1930   ate1940    ate1950    ate1960    ate1970    ate1980  \
0    maria  336477.0  749053.0  1487042.0  2476482.0  2495491.0  1616019.0   
1      ana   33395.0   56160.0   101259.0   183941.0   292835.0   421531.0   
2     joao   60155.0  141772.0   256001.0   396438.0   429148.0   279975.0   
3  gabriel    1267.0    2744.0     4256.0     6095.0     6559.0     9103.0   
4    lucas     388.0     937.0     1593.0     2787.0     4425.0     9954.0   

    ate1990   ate2000    ate2010  
0  917968.0  544296.0  1111301.0  
1  529266.0  536302.0   935169.0  
2  273960.0  352552.0   794118.0  
3   46996.0  271405.0   584024.0  
4   84484.0  517436.0   505306.0  


In [370]:
[n for n in nomes if str(n).startswith("Ross")]

['Rossana',
 'Rossini',
 'Rossano',
 'Rossine',
 'Rossane',
 'Rossi',
 'Rossandra',
 'Rossiane']

### Função para extração dos metadados

In [58]:
# list documents
def load_peek(files, n=None, verbose=False):
    result_text = dict()
    errors = list()
    
    for city, files in files.items():
        result_text[city] = {}
        
        for i, filename in enumerate(files):
            try:
                doc_id = filename.split("/")[-1].split(".")[0]
                idx = [doc_id in f for f in docs_atas[city]].index(True)
                if not idx:
                    continue
                    
                with pdfplumber.open(filename) as f:
                    print(doc_id)
                    print(f"metada: {f.metadata}")
                    print("".join([c["text"] for c in f.pages[0].chars if "bold" in c["fontname"].lower() or c["text"] == " "]))
                         
                    print("\n"*3)
            except:
                errors.append(filename)
                continue
            
    if verbose and len(errors):
        print(f"Errors found while reading:")
        for filename in errors:
            print(f"\t{filename}")
                
    return result_text

content_text = load_peek(files)

f80ced0ea740516c3d4c57ba4a7805dd
metada: {'Title': 'Processo: <<PROCESSO>> Modalidade: Pregão presencial Nº:<<AQUISIÇÃO>>', 'Author': 'Tecnologia', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20200826145423-03'00'", 'ModDate': "D:20200826145423-03'00'", 'Producer': 'Microsoft® Word 2010'}
PREFEITURA MUNICIPAL DE SÃO BENTO ABADE - ESTADO DE MINAS GERAIS CNPJ-17.877.176/0001- 29 Fone: 0** 35 3236-1213 Rua Odilon Gadbem dos Santos, nº 100 – Centro, São Bento Abade/MG Email:                       ATA REGISTRO DE PREÇOS Nº .                                                                                                                                                                1. DOS PREÇOS REGISTRADOS                                                                                                                          




c43945c062d0e37751d8850ec2c94f7c
metada: {'Author': 'Cristiane', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20200916160238-03'00'", 'ModDate':

   ATA DE REGISTRO DE PREÇOS Nº 010/2020   PROCESSO ADMINISTRATIVO LICITATÓRIO Nº 006/2020 MODALIDADE: PREGÃO PRESENCIAL Nº 005/2020– SRP Nº 0005/2020 VALIDADE 12 (DOZE) MESES  PREÂMBULO                                                                                                                                 FUTURA E EVENTUAL CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA O FORNECIMENTO DE COMBUSTÍVEIS “GASOLINA COMUM”, E “ÓLEO DIESEL S10”, OBJETIVANDO O ABASTECIMENTO DA FROTA MUNICIPAL (QUE DEVERÁ SER REALIZADO NA BOMBA DA LICITANTE VENCEDORA, NO PERÍMETRO URBANO DESTE MUNICÍPIO E DIRETAMENTE NOS VEÍCULOS AUTORIZADOS), ATRAVÉS DAS SECRETARIAS MUNICIPAIS DO MUNICÍPIO DE COQUEIRAL, ESTADO DE MINAS GERAIS, FUNDOS E CONVÊNIOS     ANEXO I – TERMO DE REFERÊNCIA E ANEXOS                            




6aa821e1e916cac34caf106284ff38b8
metada: {'Author': 'Rognei', 'Creator': 'Microsoft® Word 2016', 'CreationDate': "D:20190920072246-03'00'", 'ModDate': "D:20190920072246-03'00'", 'Producer': 'M

@icExercício: 2018Página: 1PREFEITURA MUNICIPAL DE COQUEIRAL - MG RUA MINAS GERAIS, 62 - CENTRO CNPJ: 18.239.624/0001-21    Telefone: 35 3855-1162@fc@icATA 63/2018@fcProcesso:Data:01/08/2018135Valor:106.241,57Modalidade:71 - PREGÃO - SRPAta:Vigência:6301/10/2018 à 30/09/2019Publicação:10/10/2018Valor:24.827,00Descrição:Aquisição de Medicamentos e outros3971 - 05.194.502/0001-14 - ALFALAGOS LTDA.FornecedorSolicitaçãoCódigoDescriçãoQuantidadeUnitárioTotal1661120712.000,0000600,00AGULHA HIPODÉRMICA 13X4,50,0500166112424.000,0000200,00AGULHA HIPODÉRMICA 20X5,50,0500166112434.000,0000200,00AGULHA HIPODÉRMICA 25X60,0500166112105.500,0000275,00AGULHA HIPODÉRMICA 25X70,0500166112114.000,0000200,00AGULHA HIPODÉRMICA 25X80,0500166112125.500,0000330,00AGULHA HIPODÉRMICA 40X120,060016688712,000086,00BANDEIJA RETANGULAR43,00001665999830,0000547,80CATETER P/ OXIGÊNIO TIPO ÓCULOS0,66001662609650,00007.410,00COMPRESSA DE GAZE 13 FIOS 7,5X7,5 CM11,400016626082,000071,60CUBA RIM 26X12X6CM35,800016611205

   ATA DE REGISTRO DE PREÇOS Nº 091/2020   PROCESSO ADMINISTRATIVO LICITATÓRIO Nº 099/2020 MODALIDADE: PREGÃO PRESENCIAL Nº 042/2020 – SRP Nº 041/2020 VALIDADE 12 (DOZE) MESES  PREÂMBULO                                                                                                                                                                                                                                            




5d3064bda50bd93b9f5f5628e1509116
metada: {'Author': 'Rognei', 'Creator': 'Microsoft® Word para Office 365', 'CreationDate': "D:20181010104144-03'00'", 'ModDate': "D:20181010104144-03'00'", 'Producer': 'Microsoft® Word para Office 365'}
   ATA DE REGISTRO DE PREÇOS Nº 061/2018   PROCESSO ADMINISTRATIVO LICITATÓRIO Nº 132/2017 MODALIDADE: PREGÃO PRESENCIAL Nº 076/2017 – SRP Nº 034/2017 VALIDADE 12 (DOZE) MESES  PREÂMBULO         MUNICÍPIO DE COQUEIRAL                                                                                           132/2017,    076/2017, e     

SEGUNDA ATA DO PREGÃO PRESENCIAL Nº 052/2017                                                                                                                                             3 (três) dias                                                                                              Mattheus Henrique RoganaPregoeiro OficialPablo José da SilvaEquipe de ApoioEdinamara Ribeiro SilvaEquipe de Apoio




7920f287fb2544648ce2073ebc2deccd
metada: {'Producer': 'PDFCreator 2.4.1.13', 'CreationDate': "D:20170207153425-02'00'", 'ModDate': "D:20170207153425-02'00'", 'Title': 'ATA DE REGISTRO DE PREÇO', 'Author': 'Licitação', 'Subject': '', 'Keywords': '', 'Creator': 'PDFCreator 2.4.1.13'}
ATA DE REGISTRO DE PREÇOS N.º 023/2017ATA DE REGISTRO DE PREÇOS QUE, ENTRE SI,CELEBRAM, DE UM LADO, PREFEITURA MUNICIPAL DECRISTAIS E DE OUTRO A EMPRESA RAQUEL DELOURDES SOUZA OLIVEIRA ME, NA FORMA ABAIXO.TIPO DE LICITAÇÃO: Menor Preço UnitárioPROCESSO Nº. 006/2017PREGÃO PRESENCIAL nº 006/2017 com REGISTRO

    CEP.: 36.730-000 – ESTADO DE MINAS GERAIS                            021/2019   ATA DE REGISTRO DE PREÇOS Nº 009/2019                                                                                   034/2019                                                                       1. DO OBJETO         Registro de Preços para eventual aquisição de material de consumo hospitalar em atendimento ao Fundo Municipal de Saúde,                021/2019,                2. DOS PREÇOS, ESPECIFICAÇÕES E QUANTITATIVOS                                                                                                              




88819c18e27860449b0162bef68d4a68
metada: {'Title': 'T&A EMPREENDIMENTOS', 'Author': 'Servidor', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20190702123449-03'00'", 'ModDate': "D:20190702123449-03'00'", 'Producer': 'Microsoft® Word 2010'}
    CEP.: 36.730-000 – ESTADO DE MINAS GERAIS                           PROCESSO LICITATÓRIO 023/2019.  EDITAL DE TP 003/2019. 

PREFEITURA MUNICIPAL DE PASSA VINTE                ATA DE SESSÃO PÚBLICA                                                                       Preâmbulo                  CredenciamentoFORNECEDORCPF / CNPJREPRESENTANTERegistro do PregãoLote 1 - Transporte Escolar - Contratação de empresa para prestação de Serviços de Transporte Escolar Rural linha Rio dasPedras - QUILOMETROSPROPOSTASOrdemFornecedorSituaçãoValorNegociaçãoResultado da Negociação - Lote 1 - Transporte Escolar - Contratação de empresa para prestação de Serviços de TransporteEscolar Rural linha Rio das Pedras - QUILOMETROSFornecedorValor Vencido  HabilitaçãoOcorrências




8966befe922f586c17fb62243f0119d3
metada: {'Title': 'SIPLAN RELATÓRIO - PREFEITURA MUNICIPAL DE PASSA VINTE', 'Subject': 'Relatório gerado pelo sistema SiplanWeb.', 'Keywords': 'Siplan, Siplanweb', 'Creator': 'PREFEITURA MUNICIPAL DE PASSA VINTE', 'Producer': 'TCPDF 6.2.26 (http://www.tcpdf.org)', 'CreationDate': "D:20190919092233-03'00'", 'ModDate': "D:201

# Análise do conteúdo textual

In [132]:
import nltk
from nltk.corpus import stopwords
from nltk.collocations import *

STOP_WORDS = stopwords.words('portuguese')

def drop_sw(text):
    return " ".join([wd for wd in text.split() if wd not in STOP_WORDS])

In [438]:
filter_text_pattern = re.compile("[^a-z0-9\s]")
filter_space_pattern = re.compile("\s+")

def get_plain_text(raw_text):
    plain_text = filter_text_pattern.sub("", unidecode(raw_text).lower())
    plain_text = re.sub("\s+", " ", plain_text).strip()
    return plain_text

def remove_spaces(text):
    return filter_space_pattern.sub(" ", text).strip()


def get_names(raw_text, block_words = [], mark_words = []):
    pre_processed = drop_sw(raw_text)
    tokens = pre_processed.split()
    names_found = list()
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token in nomes and token.lower() not in block_words:
            name = ""
            while token.istitle():
                name += token + " "
                i += 1
                if i == len(tokens):
                    break
                token = tokens[i]
                if token.lower() in mark_words:
                    break
            names_found.append(name.strip())
        else:
            i += 1
    
    names_found = filter(lambda name: len(name.split()) >= 3 and not name.lower().startswith("rua"), names_found)
    names_found = [re.sub("[^a-zA-Zâáãéô\s]", "", n) for n in names_found]
    return sorted(list(set(names_found)))

In [439]:
patterns = {
    "email"    : re.compile("([a-zA-Z0-9-_]+@[a-zA-Z0-9.]+)"),
    "telefone" : re.compile("((?:\(?\d{2}\)?)\s*\d{4,5}-\d{4})"),
    "cep"      : re.compile("(\d{2}\.?\d{3}-\d{3})"),
    "cpf"      : re.compile("(\d{3}\.?\d{3}\.?\d{3}-\d{2})"),
    "cnpj"     : re.compile("(\d{2}\.?\d{3}\.?\d{3}\/\d{4}-?\d{2})"),
    "date"     : re.compile("(\d{2}/\d{2}/\d{4})"),
    "month"     : re.compile("\d?\d de\s(?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\sde\s\d{4}"),
    "aquisição": re.compile("(aquisicao de.*?)(?:,|\.|\n|\s[a-z]+:|$)"),
    "valor"    : re.compile("((?:r\$\s*)?(?:\d{1,3}\.)*\d{1,3},\d{2})"),
    "rua"      : re.compile("(rua .*?),"),
    "processo" : re.compile("(processo:?\s*\d+(?:/\d+)?)"),
    "descricao" : re.compile("(descricao:.*?)(?:\.|\n|\s[a-z]+:)"),
    "contrad*s" : re.compile("(contratad[ao]s?:.*?)(?:\.|\n|\s[a-z]+:)"),
    "endereco": re.compile("(endereco:.*?)(?:\.|\n|\s[a-z]+:)"),
    "deadline": re.compile("(\d+\s*?(?:\([a-z\s]+?\))?\s*?(?:mes(?:es)?|dia(?:s)?|ano(?:s)?))"),
    "place": re.compile("([a-z\s]+?, \d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de \d{4})"),
    "processo_licitacao": re.compile("((?:processo administrativo licitatorio|processo licitatorio|processo de licitacao)[a-z\sº]*?\d{3}/\d{4})(?:\.|\n|,|;|$)")
    
}

def get_docs(full_text):
    result = {}
    for pattern in patterns:
        regex = patterns[pattern]
        matches = list(set(regex.findall(full_text)))
        print(pattern, matches)
        result[pattern] = [str(m).strip() for m in matches]
    
    return result
    

In [440]:
block_words = ["sao", "bento", "abade", "marca", "lei", "este"]
mark_words = ["presidente", "membro", "prefeita", "prestação", "pregoeiro", "representante"]

cities = {}
for city_json in os.listdir(base_path + "documentos_json"):
    city = city_json.rstrip(".json")
    cities[city] = open(base_path + "documentos_json/" + city_json)

results = []
for city, handle in cities.items():
    for document in handle:
        document = json.loads(document)
        if city not in docs_atas or document["file_id"] not in docs_atas[city]:
            continue
        full_text = " ".join(document["text_content"])
        names = get_names(full_text, block_words, mark_words)
        document_docs = get_docs(remove_spaces(unidecode(full_text.lower())))
        infos = {"city": city, "file_id": document['file_id']}
        infos["names"] = ", ".join(names)
        for doc in document_docs:
            infos[doc] = ", ".join(document_docs[doc])
            
        results.append(infos)
        
        print(f"City: {city} Document: {document['file_id']}")
        print("nomes", names)
        
        print("\n"*2)

df = pd.DataFrame.from_records(results)

email ['licitacao@saobentoabade.mg.gov.br']
telefone []
cep []
cpf ['093.670.796-80', '060.234.786-62']
cnpj ['20.352.354/0001-02', '17.877.176/0001-29']
date []
month ['17 de agosto de 2020']
aquisição ['aquisicao de materiais de laboratorio para atender a demanda do departamento de saude e meio ambiente do municipio de sao bento abade/mg']
valor ['r$124,00', 'r$672,00', 'r$3.204,00', 'r$54,60', '89,00', 'r$62,00', 'r$2.520,00', '68,00', 'r$196,00', '24,90', 'r$896,40', '47,60', 'r$408,00', '39,00', '21,80', '70,00', 'r$660,00', '55,00', 'r$960,00', 'r$2.050,00', '38,90', '49,00', 'r$675,00', '62,00', 'r$30,00', 'r$2.196,00', 'r$468,00', '61,00', '28,00', '31,00', 'r$780,00', '40,00', 'r$246,00', 'r$1.400,40', '13,50', 'r$21,80', '41,00', 'r$25.500,00', '140,00', 'r$280,00', '51,00', '4,55', 'r$17,64', '16,00', '324,45', '16,50', 'r$216,00', 'r$1.946,70', 'r$640,00', 'r$198,00', '36,00', 'r$3.600,00', 'r$224,00', '4,41', 'r$1.713,60', 'r$ 54.141,42', '130,00', '100,00']
rua ['rua odil

email []
telefone []
cep ['37.235-000']
cpf ['509.489.386-68', '482.200.946-72', '093.271.156-19', '376.391.376-91']
cnpj ['20.393.955/0001-63', '18.239.624/0001-21']
date ['03/02/2020', '06/02/2021', '07/02/2020']
month ['21 de junho de 1993', '03 de fevereiro de 2020', '17 de julho de 2002']
aquisição []
valor ['102,15', 'r$ 790.000,00', 'r$ 1.526.500,00', 'r$ 3,95', '101,14', '100,10', 'r$ 4,91', '101,10', 'r$ 736.500,00']
rua ['rua tiradentes', 'rua minas gerais', 'rua tenente antonio joao']
processo []
descricao []
contrad*s []
endereco []
deadline ['30 (trinta) dias', '05 (cinco) dias', '02 (dois) anos', '15 (quinze) dias', '12 (doze) meses', '10 (dez) dias', '30 dias', '05 (cinco) anos']
place [' coqueiral, 03 de fevereiro de 2020']
processo_licitacao ['processo administrativo licitatorio no 006/2020']
City: coqueiral Document: 1c4dec7f2fcfda38665e661919e0b124
nomes ['Clenes Ferraz Pereira']



email []
telefone []
cep ['37.235-000']
cpf ['482.200.946-72', '073.918.196-38', '013

email []
telefone []
cep ['35.661-303', '35.660-970', '37.235-000']
cpf ['093.271.156-19', '119.028.036-12', '376.391.376-91']
cnpj ['31.486.195/0001-55', '18.239.624/0001-21']
date ['29/09/2020', '28/09/2021']
month ['29 de setembro de 2020', '21 de junho de 1993', '17 de julho de 2002']
aquisição []
valor ['2,76', '137,76', '318,00', '120,00', '123,00', '30,36', '9,00', '277,20', '11,00', '5,22', '1,60', '2,00', '520,00', '207,90', '954,00', '30,80', '104,80', '110,00', '420,00', '1,31', '136,50', '5,00', '28,00', '122,00', '114,24', '522,00', '134,40', '1,89', '102,00', '2,61', '43,50', '2,18', '2.914,50', '1,12', '32,00', '582,40', '80,00', '84,00', '3,00', '20,00', '67,00', '136,64', '584,64', '174,40', '15,00', '27,30', 'r$ 8.737,70']
rua ['rua minas gerais', 'rua jurema', 'rua pedro de melo franco']
processo []
descricao []
contrad*s []
endereco []
deadline ['30 (trinta) dias', '05 (cinco) dias', '02 (dois) anos', '15 (quinze) dias', '12 (doze) meses', '10 (dez) dias', '30 dias'

email []
telefone ['35 3855-1162']
cep []
cpf []
cnpj ['08.517.373/0001-46', '18.239.624/0001-21']
date ['18/01/2019', '18/02/2020', '18/02/2019']
month []
aquisição []
valor ['2,65', '1,20', '16,52', '12,90', '400,20', '247,80', '52,20', '166,00', '209,00', '1,08', '690,00', '11,50', '247,50', '5,85', '2,28', '447,00', '8,00', '67,90', '18,86', '19,40', '67,50', '7,50', '860.282,08', '202,00', '22,50', '227,00', '52,65', '139,00', '0,66', '665,00', '16.975,00', '172,50', '1.850,00', '373,50', '2,49', '43,95', '2,43', '36,00', '0,59', '80,00', '1.625,00', '3,00', '12,00', '5,42', '8,25', '117,00', '90,00', '2,67', '15,00', '3,45', '2,27', '100,00', '13,30', '5.850,00', '150,00', '26,20', '1.864,00', '2,06', '0,75', '26.235,00', '224,00', '104,25', '390,45', '75,00', '2,52', '1.287,00', '37,00', '6,45', '8,85', '500,00', '597,00', '867,00', '4,80', '26.200,00', '1.335,00', '26,03', '504,00', '10,00', '165,15', '23,30', '9,42', '2.250,00', '1.886,00', '20,85', '9,40', '6,93', '3,51', '20

email ['admpmp@pirapetinga.mg.gov.br']
telefone []
cep ['36.730-000']
cpf []
cnpj ['17.357.099/0001-86', '18.092.825/0001-49', '20.157.057/0001-06', '19.286.643/0001-71']
date []
month []
aquisição []
valor ['r$124.281,07', 'r$118.573,51', 'r$122.764,85']
rua ['rua antonio matias']
processo []
descricao []
contrad*s []
endereco []
deadline ['05 (cinco) dias', '02 dias']
place []
processo_licitacao ['processo licitatorio 023/2019']
City: pirapetinga Document: 88819c18e27860449b0162bef68d4a68
nomes ['Adriana Batista Oliveira', 'Dirceu Oliveira Martins', 'Padua Ltda Me', 'Priscila Lemos Souza Claudia Braga Martins']



email ['admpmp@pirapetinga.mg.gov.br']
telefone []
cep ['36.730-000']
cpf []
cnpj ['18.092.825/0001-49']
date []
month ['27 de maio de 2019']
aquisição []
valor []
rua []
processo []
descricao []
contrad*s []
endereco []
deadline []
place []
processo_licitacao []
City: pirapetinga Document: b67877144aeb0a59f6e63ecf9d914bc1
nomes ['Adriana Batista Oliveira', 'Claudia Braga M

email ['pmpvinte@yahoo.com.br']
telefone ['(32) 3295-1131']
cep []
cpf []
cnpj ['18.338.210/0001-50', '17.714.200/0001-01', '33.174.625/0001-29', '17.714.200/0001-09', '28.072.149/0001-03']
date ['04/10/2019']
month ['4 de outubro de 2019']
aquisição ['aquisicao de materiais de infraestrutura para o municipio de passa vinte preambulo atadasessaopublicapregaopresencial044/2016aos04(quatro)diasdomesdeoutubrode2019']
valor ['2,65', 'r$ 24,40', '0,43', 'r$27,16', '8,31', 'r$ 4,90', 'r$ 14,08', 'r$ 81,90', '9,25', '10.500,00', 'r$ 109,75', '2,47', 'r$ 23,00', 'r$ 25,30', '0,50', '400,00', 'r$ 9.810,00', '0,69', 'r$ 79,00', 'r$ 9,00', 'r$ 2.205,00', 'r$ 23,20', '4,20', 'r$35.200,00', '36,00', 'r$ 66,80', '90,00', 'r$ 64,60', 'r$ 114,99', 'r$ 132,10', 'r$ 5,65', '150,00', 'r$ 12,00', 'r$ 10,25', 'r$ 88,27', 'r$ 111,80', '27,85', 'r$ 0,00', '3,35', 'r$ 10,18', 'r$162,99', 'r$ 5,50', 'r$ 4.752,00', 'r$ 83,00', 'r$25,50', 'r$ 0,21', 'r$ 550,00', 'r$ 688.840,00', 'r$ 72,00', 'r$ 54,88', 'r$ 96,16

In [437]:
#df.to_excel("assets_m03_extracao_informaoes_atas/output_v2.xlsx", sheet_name="resultados_previos")